In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
# from collections import Counter
import tensorflow as tf
import glob
from glob import glob
import os

sys.path.append('/home/ibecker/Dropbox/MyPythonClasses/FinalClass/')

import Classifier.Model_53.Multiband_53 as multiband
import BaseClass.plot_v2 as plot


from mpl_toolkits.axes_grid1 import make_axes_locatable
import itertools
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# To reset cuda
# sudo rmmod nvidia_uvm
# sudo modprobe nvidia_uvm
# To see if the system regognises the GPU
device = 0
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)


2023-06-22 22:15:08.331930: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-22 22:15:08.332287: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-22 22:15:08.336329: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-22 22:15:08.336518: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-22 22:15:08.336660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

### Get one of the folds, and test a classification on the test set

In [16]:
survey = 'ZTF'
# survey = 'PanStarrs'
# survey = 'Gaia'
path = os.path.join('../../CV_c/03_Train_Models/', survey, 'Results/Fold_*',)
folds = glob(path)
folds.sort()
folds

['../../CV_c/03_Train_Models/ZTF/Results/Fold_1',
 '../../CV_c/03_Train_Models/ZTF/Results/Fold_2',
 '../../CV_c/03_Train_Models/ZTF/Results/Fold_3',
 '../../CV_c/03_Train_Models/ZTF/Results/Fold_4',
 '../../CV_c/03_Train_Models/ZTF/Results/Fold_5',
 '../../CV_c/03_Train_Models/ZTF/Results/Fold_6',
 '../../CV_c/03_Train_Models/ZTF/Results/Fold_7']

In [17]:
folder_name = 'Results'
if not os.path.exists('./'+folder_name+'/'):
    os.mkdir('./'+folder_name+'/')

In [18]:
fold = folds[0]

tf.keras.backend.clear_session()
# Set the fold path
base_dir = fold+'/'

# Get the last run
path_runs_folder = os.path.join(base_dir, 'Models', '*')
path_runs = glob(path_runs_folder)
path_runs.sort()
path_run = path_runs[-1]    
    
path_preprocess = fold.replace('/03_Train_Models/', '/02_Create_Records/').replace('/'+folder_name+'/', '/Folds/')
test_files = os.path.join(path_preprocess, 'test/*.tfrecord')
path_preprocess = os.path.join(path_preprocess, 'metadata_preprocess.json')    

    
new = multiband.Network()    
settings_path = os.path.join(path_run, 'all_settings.json')

fold_name = fold.split('/')[-1]
write_path = os.path.join('./'+folder_name, fold_name+'_Results.dat')
    
# Get weights path
run = path_run.split('/')[-1]
weights_path = os.path.join(fold, 'Models', run)
new.run_test(settings_path, test_files, weights_path, df_paths = write_path);
    
#     result_path = os.path.join('./'+folder_name, fold.split('/')[-1]+'_Results.dat')
    
#     # From the results file, read the data and compute the classification scores
#     report_fold = plot.compute_classification_report(result_path)
#     reports_folds.append(report_fold)
#     # From the results file, read the data and compute the confussion matrix and the respective  labels 
#     cm_fold = plot.compute_confussion_matrices(result_path, labels=label_order)
#     cm_folds.append(cm_fold)

../../CV_c/02_Create_Records/ZTF/Folds/Fold_1/test/*.tfrecord
              precision    recall  f1-score   support

         AGN       0.68      0.62      0.65       345
      Blazar       0.82      0.60      0.69       156
     CV/Nova       0.96      0.78      0.86       109
           E       0.92      0.97      0.95      1429
         LPV       0.99      1.00      0.99      1330
         QSO       0.89      0.93      0.91      1429
         RRL       0.97      0.93      0.95      1428
        SNIa       1.00      1.00      1.00        72
         YSO       0.81      0.75      0.78       114

    accuracy                           0.92      6412
   macro avg       0.89      0.84      0.86      6412
weighted avg       0.92      0.92      0.92      6412



In [19]:
for batch in new.dataset_test:
    X = batch[0]
    y = batch[1]
    
    output = new.model_central(X)
    break

In [20]:
import pickle
with open('output_'+survey+'.pkl', 'wb') as file:
    pickle.dump([output, X, y, new.trans], file)

In [21]:
new.trans

{0: 'AGN',
 1: 'CV/Nova',
 2: 'SNIa',
 3: 'LPV',
 4: 'RRL',
 5: 'QSO',
 6: 'E',
 7: 'YSO',
 8: 'Blazar'}